# Predicting more than one day into the future

Without extra work, technicals only offer an opportunity to forecast tomorrow's closing price because TI information doesn't exist yet for days further into the future. I propose to impute future TI predictor information based, not on actual closing prices, but instead on previously forecasted values.

In [5]:
cd ..

/home/jovyan/Dropbox/Public/dsi-plus/s_p500


In [29]:
import pickle
from lib.utils import extract_technicals

import warnings
warnings.filterwarnings('ignore')

In [21]:
with open("lib/tech_func_tools.pkl", "rb") as file:
    tech_funcs_dict, tech_funcs_params = pickle.load(file)
    
#load an example technicals df
apple = pd.read_csv("data/sandp500/individual_stocks_5yr_TECHNICALS/AAPL_data.csv")

In [68]:
def update_technicals(df, close_price, tech_funcs_dict, tech_funcs_params):
    '''
    Takes technicals df adds a new close for the next day, then updates the next day's 
    technicals based on that close price
    '''
    seed_columns = ['Open', 'High', 'Low', 'Close', 'Volume',]
    seed_columns_df = df[seed_columns]
    
    last_row = seed_columns_df.iloc[-1, :].copy()
    last_row["Close"] = close_price
    seed_columns_df = seed_columns_df.append(last_row, ignore_index = True)
    
    return extract_technicals(seed_columns_df, tech_funcs_dict, tech_funcs_params)

In [74]:
update_technicals(apple, 4000, tech_funcs_dict, tech_funcs_params).tail()

,Open,High,Low,Close,Volume,Acc/Dist_ROC_1,ADX_14_50,ATR_14,BollingerB_5,Bollinger%b_5,...,RSI_6,RSI_12,STD_5,STD_20,SO%d_5,SO%d_20,SO%k,Trix_5,Trix_20,TSI_25_13
1254,158.60,161.83,158.27,160.08,36205896.0,0.0,0.373994,2.986480,0.046474,0.838875,...,0.873646,0.784644,1.831057,3.823337,0.559125,0.539418,0.508427,0.005674,0.000839,0.276910
1255,159.26,161.27,159.11,161.06,26131530.0,0.0,0.363050,2.876283,0.059382,0.784742,...,0.873646,0.784644,2.351249,4.042700,0.673676,0.574024,0.902778,0.006351,0.001007,0.306047
1256,159.90,160.00,154.63,155.32,40804273.0,-0.0,0.352535,3.350112,0.061325,0.189794,...,0.407591,0.532323,2.427420,3.907066,0.491948,0.531592,0.128492,0.005146,0.001145,0.270984
1257,156.60,158.57,156.07,157.48,26257096.0,0.0,0.342433,3.336763,0.056815,0.381218,...,0.407591,0.532323,2.252021,3.902345,0.515965,0.534679,0.564000,0.004037,0.001270,0.256990
1258,156.60,158.57,156.07,4000.00,26257096.0,0.0,0.342433,3.225195,7.414767,0.947213,...,0.407591,0.532323,1717.979205,860.062586,512.867977,146.919186,1537.572000,0.917455,0.023703,0.973431


Update technicals has been added to `lib/utils.py`

### Problem

A lot of technicals require High, Low, and Volume values, on top of Close. Our model only predicts close. The `update_technicals` function just uses yesterday's values for the three non-predicted values. I'm not sure if that is a naive assumption or not. Yeah actually I think that's pretty bad.

This seems like a coding-heavy endeavor, and I don't even think it will work very well. I'm gonna put it on hold for now, unless it sees important.

## Using `update_technicals` for forward prediction